In [6]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [7]:
load_dotenv(override=True)

True

In [8]:
class State(TypedDict):
    
    messages: Annotated[list, add_messages]
    flagged_ingredients: List[str]
    safety_score: int


graph_builder = StateGraph(State)

In [9]:


def flag_ingredient(ingredient_name: str, reason: str):
    """Flags a harmful ingredient and records the scientific reason.
        Use this to display warnings on the user's dashboard."""
    return f"FLAGGED: {ingredient_name}. REASON: {reason}"

tool_flag = Tool(
        name="flag_harmful_ingredient",
        func=flag_ingredient,
        description="Useful for marking specific ingredients as dangerous to be displayed on the UI."
    )